### Importing all the needed packages and libraries

In [ ]:
import keras
import cv2
import numpy as np
import argparse
import os
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
tf.config.list_physical_devices('GPU') 
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.layers import AveragePooling2D, Dropout,Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix,plot_confusion_matrix
import seaborn as sn
import os
import itertools
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Add, Activation, Multiply, concatenate
from tensorflow.keras.applications import VGG16,DenseNet201,ResNet50,VGG19,Xception,InceptionResNetV2,InceptionV3
from tensorflow.keras.layers import AveragePooling2D,MaxPooling2D,GlobalAveragePooling2D

### Defining the paths

In [ ]:
train_path='../input/newdataset/final_dataset/train'
test_path='../input/newdataset/final_dataset/test'
val_path='../input/newdataset/final_dataset/val'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

### Data Augmentation

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path,target_size = (320, 320),shuffle=True,seed=42,class_mode="categorical",color_mode = 'rgb',batch_size = 16)
test_generator = test_datagen.flow_from_directory(test_path,target_size = (320, 320),color_mode = 'rgb',batch_size = 1,seed=42,class_mode="categorical",shuffle = False)
val_generator = test_datagen.flow_from_directory(val_path,target_size = (320, 320),color_mode = 'rgb',batch_size = 1,seed=42,class_mode="categorical",shuffle = False)

### Plotting of Sample Images

In [ ]:
def plot_img(img_arr):
  fig,axes=plt.subplots(2,8,figsize=(10,10))
  axes=axes.flatten()
  for img,ax in zip(img_arr,axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
imgs,labels=next(train_generator)

In [ ]:
plot_img(imgs)
print(labels)

In [ ]:
#RANDOM INITIALIZATION OF WEIGHTS
class_weight = {
    0:0.85,
    1:0.15
}

### ResNet50

In [ ]:
lr = 1e-4
EPOCHS = 100
BS = 16
k_fold=2
cv_scores, model_history = list(), list()
for i in range(k_fold):
    
    print("K-FOLD:",i+1)
    baseModel = ResNet50(include_top=False,weights='imagenet',input_tensor=Input(shape=(320, 320, 3)))
    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(4, 4))(headModel)#pool_size=(4, 4)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.3)(headModel)
    headModel = Dense(2, activation="softmax")(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)
    print("[INFO] compiling model...")
    opt = Adam(lr=lr,decay=lr/EPOCHS)
    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    print("[INFO] training head...")
    H = model.fit(train_generator,
                    steps_per_epoch = 320//BS,
                    epochs = EPOCHS,
                    validation_data = val_generator,
                    validation_steps = 26,
                   class_weight=class_weight)
    predictions=model.predict(test_generator,use_multiprocessing=True)
    predIdxs = np.argmax(predictions, axis=1)
    cm=confusion_matrix(y_true=test_generator.classes,y_pred=predIdxs)
    print(cm)
    print(classification_report(test_generator.classes,predIdxs))
    total = sum(sum(cm))
    acc = (cm[0, 0] + cm[1, 1]) / total
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    # show the confusion matrix, accuracy, sensitivity, and specificity
    print(cm)
    print("acc: {:.4f}".format(acc))
    print("sensitivity: {:.4f}".format(sensitivity))
    print("specificity: {:.4f}".format(specificity))

    cv_scores.append(acc)
    model_history.append(H.history)

print('Estimated Accuracy %.3f (%.3f)' % (np.mean(cv_scores), np.std(cv_scores)))

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_generator.classes, predIdxs)

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for COVID19 classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

### VGG19

In [ ]:
lr = 1e-4
EPOCHS = 100
BS = 16
k_fold=2
cv_scores, model_history = list(), list()
for i in range(k_fold):
    
    print("K-FOLD:",i+1)
    baseModel = VGG19(include_top=False,weights='imagenet',input_tensor=Input(shape=(320, 320, 3)))
    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(4, 4))(headModel)#pool_size=(4, 4)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.3)(headModel)
    headModel = Dense(2, activation="softmax")(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)
    print("[INFO] compiling model...")
    opt = Adam(lr=lr,decay=lr/EPOCHS)
    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    print("[INFO] training head...")
    H = model.fit(train_generator,
                    steps_per_epoch = 320//BS,
                    epochs = EPOCHS,
                    validation_data = val_generator,
                    validation_steps = 26,
                   class_weight=class_weight)
    predictions=model.predict(test_generator,use_multiprocessing=True)
    predIdxs = np.argmax(predictions, axis=1)
    cm=confusion_matrix(y_true=test_generator.classes,y_pred=predIdxs)
    print(cm)
    print(classification_report(test_generator.classes,predIdxs))
    total = sum(sum(cm))
    acc = (cm[0, 0] + cm[1, 1]) / total
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    # show the confusion matrix, accuracy, sensitivity, and specificity
    print(cm)
    print("acc: {:.4f}".format(acc))
    print("sensitivity: {:.4f}".format(sensitivity))
    print("specificity: {:.4f}".format(specificity))

    cv_scores.append(acc)
    model_history.append(H.history)

print('Estimated Accuracy %.3f (%.3f)' % (np.mean(cv_scores), np.std(cv_scores)))

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_generator.classes, predIdxs)

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for COVID19 classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

### VGG16

In [ ]:
lr = 1e-4
EPOCHS = 100
BS = 16
k_fold=2
cv_scores, model_history = list(), list()
for i in range(k_fold):
    
    print("K-FOLD:",i+1)
    baseModel = VGG16(include_top=False,weights='imagenet',input_tensor=Input(shape=(320, 320, 3)))
    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(4, 4))(headModel)#pool_size=(4, 4)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.3)(headModel)
    headModel = Dense(2, activation="softmax")(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)
    print("[INFO] compiling model...")
    opt = Adam(lr=lr,decay=lr/EPOCHS)
    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    print("[INFO] training head...")
    H = model.fit(train_generator,
                    steps_per_epoch = 320//BS,
                    epochs = EPOCHS,
                    validation_data = val_generator,
                    validation_steps = 26,
                   class_weight=class_weight)
    predictions=model.predict(test_generator,use_multiprocessing=True)
    predIdxs = np.argmax(predictions, axis=1)
    cm=confusion_matrix(y_true=test_generator.classes,y_pred=predIdxs)
    print(cm)
    print(classification_report(test_generator.classes,predIdxs))
    total = sum(sum(cm))
    acc = (cm[0, 0] + cm[1, 1]) / total
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    # show the confusion matrix, accuracy, sensitivity, and specificity
    print(cm)
    print("acc: {:.4f}".format(acc))
    print("sensitivity: {:.4f}".format(sensitivity))
    print("specificity: {:.4f}".format(specificity))

    cv_scores.append(acc)
    model_history.append(H.history)

print('Estimated Accuracy %.3f (%.3f)' % (np.mean(cv_scores), np.std(cv_scores)))

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_generator.classes, predIdxs)

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for COVID19 classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

### Xception

In [ ]:
lr = 1e-4
EPOCHS = 100
BS = 16
k_fold=2
cv_scores, model_history = list(), list()
for i in range(k_fold):
    
    print("K-FOLD:",i+1)
    baseModel = Xception(include_top=False,weights='imagenet',input_tensor=Input(shape=(320, 320, 3)))
    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(4, 4))(headModel)#pool_size=(4, 4)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.3)(headModel)
    headModel = Dense(2, activation="softmax")(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)
    print("[INFO] compiling model...")
    opt = Adam(lr=lr,decay=lr/EPOCHS)
    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    print("[INFO] training head...")
    H = model.fit(train_generator,
                    steps_per_epoch = 320//BS,
                    epochs = EPOCHS,
                    validation_data = val_generator,
                    validation_steps = 26,
                   class_weight=class_weight)
    predictions=model.predict(test_generator,use_multiprocessing=True)
    predIdxs = np.argmax(predictions, axis=1)
    cm=confusion_matrix(y_true=test_generator.classes,y_pred=predIdxs)
    print(cm)
    print(classification_report(test_generator.classes,predIdxs))
    total = sum(sum(cm))
    acc = (cm[0, 0] + cm[1, 1]) / total
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    # show the confusion matrix, accuracy, sensitivity, and specificity
    print(cm)
    print("acc: {:.4f}".format(acc))
    print("sensitivity: {:.4f}".format(sensitivity))
    print("specificity: {:.4f}".format(specificity))

    cv_scores.append(acc)
    model_history.append(H.history)

print('Estimated Accuracy %.3f (%.3f)' % (np.mean(cv_scores), np.std(cv_scores)))

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_generator.classes, predIdxs)

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for COVID19 classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

### DenseNet201

In [ ]:
lr = 1e-4
EPOCHS = 100
BS = 16
k_fold=2
cv_scores, model_history = list(), list()
for i in range(k_fold):
    
    print("K-FOLD:",i+1)
    baseModel = DenseNet201(include_top=False,weights='imagenet',input_tensor=Input(shape=(320, 320, 3)))
    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(4, 4))(headModel)#pool_size=(4, 4)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.3)(headModel)
    headModel = Dense(2, activation="softmax")(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)
    print("[INFO] compiling model...")
    opt = Adam(lr=lr,decay=lr/EPOCHS)
    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    print("[INFO] training head...")
    H = model.fit(train_generator,
                    steps_per_epoch = 320//BS,
                    epochs = EPOCHS,
                    validation_data = val_generator,
                    validation_steps = 26,
                   class_weight=class_weight)
    predictions=model.predict(test_generator,use_multiprocessing=True)
    predIdxs = np.argmax(predictions, axis=1)
    cm=confusion_matrix(y_true=test_generator.classes,y_pred=predIdxs)
    print(cm)
    print(classification_report(test_generator.classes,predIdxs))
    total = sum(sum(cm))
    acc = (cm[0, 0] + cm[1, 1]) / total
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    # show the confusion matrix, accuracy, sensitivity, and specificity
    print(cm)
    print("acc: {:.4f}".format(acc))
    print("sensitivity: {:.4f}".format(sensitivity))
    print("specificity: {:.4f}".format(specificity))

    cv_scores.append(acc)
    model_history.append(H.history)

print('Estimated Accuracy %.3f (%.3f)' % (np.mean(cv_scores), np.std(cv_scores)))

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_generator.classes, predIdxs)

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for COVID19 classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

### InceptionResNetV2

In [ ]:
lr = 1e-4
EPOCHS = 100
BS = 16
k_fold=2
cv_scores, model_history = list(), list()
for i in range(k_fold):
    
    print("K-FOLD:",i+1)
    baseModel = InceptionResNetV2(include_top=False,weights='imagenet',input_tensor=Input(shape=(320, 320, 3)))
    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(4, 4))(headModel)#pool_size=(4, 4)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.3)(headModel)
    headModel = Dense(2, activation="softmax")(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)
    print("[INFO] compiling model...")
    opt = Adam(lr=lr,decay=lr/EPOCHS)
    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    print("[INFO] training head...")
    H = model.fit(train_generator,
                    steps_per_epoch = 320//BS,
                    epochs = EPOCHS,
                    validation_data = val_generator,
                    validation_steps = 26,
                   class_weight=class_weight)
    predictions=model.predict(test_generator,use_multiprocessing=True)
    predIdxs = np.argmax(predictions, axis=1)
    cm=confusion_matrix(y_true=test_generator.classes,y_pred=predIdxs)
    print(cm)
    print(classification_report(test_generator.classes,predIdxs))
    total = sum(sum(cm))
    acc = (cm[0, 0] + cm[1, 1]) / total
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    # show the confusion matrix, accuracy, sensitivity, and specificity
    print(cm)
    print("acc: {:.4f}".format(acc))
    print("sensitivity: {:.4f}".format(sensitivity))
    print("specificity: {:.4f}".format(specificity))

    cv_scores.append(acc)
    model_history.append(H.history)

print('Estimated Accuracy %.3f (%.3f)' % (np.mean(cv_scores), np.std(cv_scores)))

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_generator.classes, predIdxs)

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for COVID19 classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)